# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv("scraped/hm.csv")



In [3]:
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [4]:
#extracting article id
df['article_id'] = df.url.str.extract("(\d\d\d\d\d-\w)")
df['article_id']

0     13472-N
1     35079-D
2     72462-A
3     58926-C
4     74242-A
5     70965-D
6     62818-B
7     69163-B
8     40910-C
9     69699-B
10    76916-A
11    68473-C
12    74498-A
13    41512-E
14    76727-A
15    76977-A
16    73369-A
17    59417-D
18    76797-A
19    76794-A
20    80432-A
21    78352-A
22    70658-A
23    72111-A
24    77665-A
25    70603-A
26    72262-B
27    70427-A
28    72262-A
29    77665-B
30    73380-A
31    35079-C
32    74773-A
33    75033-A
34    60637-C
35    70306-A
36    60621-B
37    69643-A
38    72455-B
39    72460-B
40    72462-B
41    71871-A
42    73734-A
43    74267-A
44    73344-A
45    73325-C
46    74264-A
47    72455-A
48    70799-B
49    68473-D
50    72111-C
51    58926-B
52    58211-A
53    75469-A
54    76982-A
55    74825-A
56    74263-A
57    78047-A
58    60637-G
59    78381-A
Name: article_id, dtype: object

In [5]:
df['price'].head()

0      $59.99 $129
1     $6.99 $17.99
2     $7.99 $17.99
3     $6.99 $12.99
4    $17.99 $24.99
Name: price, dtype: object

In [6]:
#extracting original price
df['price_original'] = df.price.str.extract(" \$(\d\d?\.?\d?\d?)")
df['price_original'] = pd.to_numeric(df['price_original'])
df['price_original'].dtype

dtype('float64')

In [7]:
#extracting discounted price
df['price_discounted'] = df.price.str.extract("\$(\d\d?\.?\d?\d?)")
df['price_discounted'] = pd.to_numeric(df['price_discounted'])
df['price_discounted'].dtype

dtype('float64')

In [8]:
#calculating the discount percentage
df['pct_discount'] = round((df['price_original'] - df['price_discounted']) / df['price_original'] * 100) 
df['pct_discount'].head()

0    53.0
1    61.0
2    56.0
3    46.0
4    28.0
Name: pct_discount, dtype: float64

In [9]:
df.head()

,name,price,url,article_id,price_original,price_discounted,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472-N,129.00,59.99,53.0
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079-D,17.99,6.99,61.0
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462-A,17.99,7.99,56.0
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926-C,12.99,6.99,46.0
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242-A,24.99,17.99,28.0


In [10]:
#dropping the old price column
df.drop('price', axis=1, inplace=True)


In [11]:
df.head()

,name,url,article_id,price_original,price_discounted,pct_discount
0,Washed Linen Duvet Cover Set,http://www.hm.com/us/product/13472?article=13472-N,13472-N,129.00,59.99,53.0
1,Candle in Glass Jar,http://www.hm.com/us/product/35079?article=35079-D,35079-D,17.99,6.99,61.0
2,Glittery Cushion Cover,http://www.hm.com/us/product/72462?article=72462-A,72462-A,17.99,7.99,56.0
3,Textured-weave Cushion Cover,http://www.hm.com/us/product/58926?article=58926-C,58926-C,12.99,6.99,46.0
4,Stoneware Bowl,http://www.hm.com/us/product/74242?article=74242-A,74242-A,24.99,17.99,28.0


In [12]:
#saving as csv
df.to_csv('hm_cleaned.csv', index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [13]:
df2 = pd.read_csv("scraped/sci-fi.csv")

In [14]:
df2

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 people voted\n \n \n",6,"\nDivergent (Divergent, #1)\n",/book/show/13335037-divergent
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 people voted\n \n \n",7,"\nCatching Fire (The Hunger Games, #2)\n",/book/show/6148028-catching-fire
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 people voted\n \n \n",8,"\nThe Giver (The Giver, #1)\n",/book/show/3636.The_Giver
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 people voted\n \n \n",9,\nThe Dispossessed\n,/book/show/13651.The_Dispossessed
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people voted\n \n \n",10,\nKindred\n,/book/show/60931.Kindred


In [15]:
#extracting series number
df2['series_no'] = df2.title.str.extract(", (#\d)\)")
df2['series_no'].head()

0    NaN
1     #1
2    NaN
3     #1
4    NaN
Name: series_no, dtype: object

In [16]:
#extracting series name
df2['series'] = df2.title.str.extract("\((.*), ")
df2['series'].head()

0                          NaN
1             The Hunger Games
2                          NaN
3    A Wrinkle in Time Quintet
4                          NaN
Name: series, dtype: object

In [17]:
#extracting score votes
df2['score_votes'] = df2.full_score.str.extract("(\d?\d?\d?) p")
df2['score_votes'].head()

0    292
1    282
2    205
3    185
4    165
Name: score_votes, dtype: object

In [18]:
#extracting total score
df2['total_score'] = df2.full_score.str.extract(": (\d?\d?\,?\d?\d?\d?)")
df2['total_score'].head() 

0    28,539
1    27,566
2    20,049
3    17,684
4    16,070
Name: total_score, dtype: object

In [19]:
#extracting rating count
df2['rating_count'] = df2.full_rating.str.extract("(\d?\d?\d?\,?\d?\d?\d?) ratings")
df2['rating_count'].head()

0    785,502
1    212,935
2    922,308
3    702,272
4     77,664
Name: rating_count, dtype: object

In [20]:
#extracting average rating
df2['avg_rating'] = df2.full_rating.str.extract("(\d\.\d\d)")
df2['avg_rating'].head()

0    4.07
1    4.34
2    3.76
3    4.04
4    4.06
Name: avg_rating, dtype: object

In [21]:
#extracting title
df2['title'] = df2['title'].str.strip()
df2['title'].head()

0                                  The Handmaid's Tale
1              The Hunger Games (The Hunger Games, #1)
2               Frankenstein, or The Modern Prometheus
3    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                            The Left Hand of Darkness
Name: title, dtype: object

In [22]:
df2.head()

,full_rating,full_score,rank,title,url,series_no,series,score_votes,total_score,rating_count,avg_rating
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN,NaN,292,"28,539","785,502",4.07
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,#1,The Hunger Games,282,"27,566","212,935",4.34
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,NaN,NaN,205,"20,049","922,308",3.76
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,#1,A Wrinkle in Time Quintet,185,"17,684","702,272",4.04
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN,NaN,165,"16,070","77,664",4.06


In [23]:
df2.drop('full_rating', axis=1, inplace=True)


In [24]:
df2.drop('full_score', axis=1, inplace=True)


In [25]:
df2.head()

,rank,title,url,series_no,series,score_votes,total_score,rating_count,avg_rating
0,1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN,NaN,292,"28,539","785,502",4.07
1,2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,#1,The Hunger Games,282,"27,566","212,935",4.34
2,3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,NaN,NaN,205,"20,049","922,308",3.76
3,4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,#1,A Wrinkle in Time Quintet,185,"17,684","702,272",4.04
4,5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN,NaN,165,"16,070","77,664",4.06


In [49]:
df.to_csv('sci-fi_cleaned.csv', index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [26]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df3 = pd.DataFrame({'filename': filenames, 'content': contents})

In [27]:

pd.set_option('display.max_colwidth', -1)





In [28]:
#extracting access info
df3['access'] = df3.content.str.extract(r"information is (\w+)")
df3['access'] 
                    

0     NaN    
1     NaN    
2     granted
3     NaN    
4     denied 
5     denied 
6     denied 
7     denied 
8     granted
9     denied 
10    granted
11    NaN    
12    denied 
13    granted
14    granted
Name: access, dtype: object

In [29]:
#extracting judge name
df3['judge'] = df3.content.str.extract(r"(\w+, \w+ ?\w?.?), Admin")
df3['judge']

0     MENDEZ, Francisco     
1     NaN                   
2     LOUGHRAN, Edward W.   
3     DUFFY, James F.       
4     NaN                   
5     HOWE, Philip S.       
6     COACHER, Robert E.    
7     DUFFY, James F.       
8     NaN                   
9     GOLDSTEIN, Jennifer I.
10    HOGAN, Erin C.        
11    NaN                   
12    MOGUL, Martin H.      
13    HARVEY, Mark          
14    NaN                   
Name: judge, dtype: object

In [30]:
#extracting applicant's lawyer
df3['lawyer_app'] = df3.content.str.extract(r"cant: (.*)") 
df3['lawyer_app'] = df3.content.str.extract(r"cant: (.*)") 
df3['lawyer_app']

0     Pro se                  
1     NaN                     
2     Mark S. Zaid, Esq.      
3     Pro se                  
4     Pro se                  
5     Pro se                  
6     Pro se                  
7     Pro se                  
8     Ryan C. Nerney, Esquire 
9     Pro Se                  
10    Pro se                  
11    NaN                     
12    Pro se                  
13    Mark S. Zaid, Esq.      
14    Stephen Glassman, Esq.  
Name: lawyer_app, dtype: object

In [31]:
#extracting government's lawyer
df3['lawyer_gov'] = df3.content.str.extract("ment: (.*), E")                                            
df3['lawyer_gov']                                     

0     David F. Hayes     
1     NaN                
2     Robert J. Kilmartin
3     Richard Stevens    
4     Julie R. Mendez    
5     David F. Hayes     
6     Stephanie C. Hess  
7     Tara Karoian       
8     Richard Stevens    
9     Robert Kilmartin   
10    Eric Borgstrom     
11    NaN                
12    Tovah Minster      
13    Julie R. Mendez    
14    Erin Thompson      
Name: lawyer_gov, dtype: object

Okay, now do the work and **make those new columns!**

In [32]:
df3.head()

filename  \
0  pdfs/11-11916.h1.pdf.txt   
1  pdfs/12-01601.a1.pdf.txt   
2  pdfs/11-03073.h1.pdf.txt   
3  pdfs/11-04909.h1.pdf.txt   
4  pdfs/11-08313.h1.pdf.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [33]:
austen = open("data/Austen_Pride.txt", encoding="utf8").read()
df4 = pd.DataFrame({'austen': [austen]})


## Look at the first 500 or so characters of it 

In [34]:
pd.set_option("display.max_colwidth", 500)
df4

,austen
0,"Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard t..."


## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [35]:
#finding she or he
he_or_she = re.findall(r"\bs?he\b", austen, re.IGNORECASE)
len(he_or_she)

3047

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [36]:
#finding she or he and the following word
he_she_plus = re.findall(r"\bs?he\b \w+", austen, re.IGNORECASE)
he_she_plus

['he is',
 'he had',
 'she told',
 'he came',
 'he agreed',
 'he is',
 'he married',
 'he may',
 'he comes',
 'she ought',
 'he comes',
 'he chooses',
 'she is',
 'She was',
 'she was',
 'she fancied',
 'He had',
 'he should',
 'she had',
 'he suddenly',
 'She has',
 'She is',
 'she times',
 'she will',
 'she will',
 'he continued',
 'he wished',
 'she began',
 'she had',
 'he spoke',
 'he left',
 'he would',
 'he eluded',
 'He was',
 'he meant',
 'He had',
 'he had',
 'he saw',
 'he wore',
 'She could',
 'he could',
 'she began',
 'he might',
 'he ought',
 'he brought',
 'he had',
 'he was',
 'he was',
 'he was',
 'he was',
 'He was',
 'he would',
 'She is',
 'he looked',
 'he withdrew',
 'She is',
 'She told',
 'she had',
 'she had',
 'he was',
 'he had',
 'He had',
 'he had',
 'she entered',
 'she looked',
 'he actually',
 'she was',
 'he asked',
 'he asked',
 'he did',
 'he seemed',
 'she was',
 'he inquired',
 'she was',
 'he danced',
 'he had',
 'he would',
 'he had',
 'He is',
 

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [37]:
#capturing pronouns and following word in separate capture groups
re.findall(r"(\bs?he)\b (\w+)", austen, re.IGNORECASE)


[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'times'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'is'),
 ('She', 't

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [38]:
matches2 = re.findall(r"(\bs?he)\b (\w+)", austen, re.IGNORECASE)
df5 = pd.DataFrame(matches2, columns=['pronoun','verb'])

df5

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed
5,he,is
6,he,married
7,he,may
8,he,comes
9,she,ought


## How many times is each pronoun used?

In [39]:
df5.pronoun.value_counts()


she    1322
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [40]:
df5['pronoun'] = df5['pronoun'].str.lower()

df5.pronoun.value_counts()



she    1647
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [41]:
df5.verb.value_counts()

was               372
had               371
could             172
is                139
would              94
has                70
did                67
will               50
might              46
should             41
felt               38
must               37
said               33
saw                32
thought            31
added              31
then               26
replied            22
continued          21
looked             21
does               20
came               19
spoke              19
found              19
may                18
went               16
soon               16
knew               15
never              14
began              14
                 ... 
valued              1
bowed               1
detained            1
raised              1
herself             1
companion           1
disliked            1
promised            1
owns                1
eluded              1
hastened            1
improved            1
coming              1
persuaded           1
frightened

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [42]:
#top20 for she
shes = df5[df5['pronoun'] == 'she']
shes.verb.value_counts().head(20)


was          212
had          205
could        132
is            65
would         59
did           38
felt          33
saw           29
will          26
might         25
added         23
has           21
said          20
thought       18
then          15
should        15
found         13
must          13
soon          12
continued     11
Name: verb, dtype: int64

In [43]:
#top20 for he
hes = df5[df5['pronoun'] == 'he']
hes.verb.value_counts().head(20)


had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
must        24
will        24
might       21
replied     14
thought     13
may         13
came        13
said        13
does        12
never       12
meant       11
then        11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [44]:
she_cried = shes[shes['verb'] == 'cried']
#she_cried.verb.value_counts()

he_cried = hes[hes['verb'] == 'cried']
#he_cried.verb.value_counts()

#Another way to find out how many women and men are crying:
#she_cried = re.findall(r"(\bshe\b cried)", austen, re.IGNORECASE)
#she_cried
#he_cried = re.findall(r"(\bhe\b cried)", austen, re.IGNORECASE)
#he_cried


perc_she_cries = round((len(she_cried)/(len(she_cried) + len(he_cried))*100))

print("Out of crying people", perc_she_cries, "percent are women.")

Out of crying people 92 percent are women.


## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [45]:
tolkien = open("data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt", encoding="utf8").read()
df6 = pd.DataFrame({'tolkien': [tolkien]})

In [46]:
matches3 = re.findall(r"(\bs?he)\b", tolkien, re.IGNORECASE)
df7 = pd.DataFrame(matches3, columns=['pronoun'])


In [47]:
df7['pronoun'] = df7['pronoun'].str.lower()

df_heshe_tolkien = round(df7.pronoun.value_counts(normalize = True), 2)*100

df_heshe_austen = round(df5.pronoun.value_counts(normalize = True), 2)*100
df_heshe_austen

she    56.0
he     44.0
Name: pronoun, dtype: float64

In [48]:
print("In Fellowship of the Ring there are", df_heshe_tolkien.he, "percent of 'he' and", df_heshe_tolkien.she, "percent of 'she', whereas in Pride and Prejudice there are", df_heshe_austen.he, "percent of 'he' and", df_heshe_austen.she, "percent of 'she'")

In Fellowship of the Ring there are 95.0 percent of 'he' and 5.0 percent of 'she', whereas in Pride and Prejudice there are 44.0 percent of 'he' and 56.00000000000001 percent of 'she'
